In [6]:
import rpy2.robjects as robjects
import pandas as pd
import numpy as np
import os

In [7]:
# Define the path to the data folder
data_path = "sessions"

# Function to read RDS file
def read_rds(file_path):
    readRDS = robjects.r['readRDS']
    return readRDS(file_path)

# load all sessions
sessions = []
for i in range(1, 19):
    file_name = f'session{i}.rds'
    file_path = os.path.join(data_path, file_name)
    session_data = read_rds(file_path)
    sessions.append(session_data)

In [8]:
# Initialize a list to store session summaries
session_summaries = []

# Process each session
for i in sessions:
    # Extract session metadata
    mouse_name = i.rx2('mouse_name')[0]
    date_exp = i.rx2('date_exp')[0]
    
    # Extract neural data
    spks = i.rx2('spks')  # Spike trains (list of matrices)
    # Convert R matrix to numpy array for the first trial
    first_trial_spks = np.array(spks[0])
    n_neurons = first_trial_spks.shape[0]  # Number of neurons (rows in the spike matrix)
    n_trials = len(spks)  # Number of trials (length of the list)
    # Extract feedback data
    feedback_type = np.array(i.rx2('feedback_type'))  # Feedback type (1 or -1)
    success_rate = np.mean(feedback_type == 1)  # Success rate
    
    # Extract brain area data
    brain_area = np.array(i.rx2('brain_area'))  # Convert to numpy array
    n_brain_area = len(np.unique(brain_area))  # Number of unique brain areas
    
    # Append session summary to the list
    session_summaries.append({
        "mouse_name": mouse_name,
        "date_exp": date_exp,
        "n_brain_area": n_brain_area,
        "n_neurons": n_neurons,
        "n_trials": n_trials,
        "success_rate": success_rate
    })

# Convert to DataFrame for easier analysis
session_summary_df = pd.DataFrame(session_summaries)

In [14]:
session_summary_df.head()

,mouse_name,date_exp,n_brain_area,n_neurons,n_trials,success_rate
0,Cori,2016-12-14,8,734,114,0.605263
1,Cori,2016-12-17,5,1070,251,0.633466
2,Cori,2016-12-18,11,619,228,0.662281
3,Forssmann,2017-11-01,11,1769,249,0.666667
4,Forssmann,2017-11-02,10,1077,254,0.661417
